## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3061. Calculate Trapping Rain Water (Hard)**

**Table: Heights**


| Column Name | Type |
|-------------|------|
| id          | int  |
| height      | int  |

id is the primary key (column with unique values) for this table, and it is guaranteed to be in sequential order.
Each row of this table contains an id and height.

**Write a solution to calculate the amount of rainwater can be trapped between the bars in the landscape, considering that each bar has a width of 1 unit.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:**
**Heights table:**
| id  | height |
|-----|--------|
| 1   | 0      |
| 2   | 1      |
| 3   | 0      |
| 4   | 2      |
| 5   | 1      |
| 6   | 0      |
| 7   | 1      |
| 8   | 3      |
| 9   | 2      |
| 10  | 1      |
| 11  | 2      |
| 12  | 1      |

**Output:**
| total_trapped_water | 
|---------------------|
| 6                   | 

**Explanation:**

![](https://fastly.jsdelivr.net/gh/doocs/leetcode@main/solution/3000-3099/3061.Calculate%20Trapping%20Rain%20Water/images/trapping_rain_water.png)

The elevation map depicted above (in the black section) is graphically represented with the x-axis denoting the id and the y-axis representing the heights [0,1,0,2,1,0,1,3,2,1,2,1]. In this scenario, 6 units of rainwater are trapped within the blue section.

In [0]:
heights_data_3061 = [
    (1, 0), (2, 1), (3, 0), (4, 2), (5, 1), (6, 0), (7, 1), 
    (8, 3), (9, 2), (10, 1), (11, 2), (12, 1)
]

heights_columns_3061 = ["id", "height"]
heights_df_3061 = spark.createDataFrame(heights_data_3061, heights_columns_3061)
heights_df_3061.show()

+---+------+
| id|height|
+---+------+
|  1|     0|
|  2|     1|
|  3|     0|
|  4|     2|
|  5|     1|
|  6|     0|
|  7|     1|
|  8|     3|
|  9|     2|
| 10|     1|
| 11|     2|
| 12|     1|
+---+------+



In [0]:
w_left = Window.orderBy("id").rowsBetween(Window.unboundedPreceding, 0)

In [0]:
w_right = Window.orderBy(desc("id")).rowsBetween(Window.unboundedPreceding, 0)

In [0]:
heights_df_3061 = heights_df_3061\
                    .withColumn("max_left", max("height").over(w_left))\
                        .withColumn("max_right", max("height").over(w_right))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
heights_df_3061\
    .withColumn( "water",
                when(
                    least("max_left", "max_right") - col("height") > 0,
                    least("max_left", "max_right") - col("height")
                    ).otherwise(0)
                ).agg(sum("water").alias("total_trapped_water")).display()

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


total_trapped_water
6
